In [6]:
######################## PyTorch Implementation of Linear Regression with a Toy Problem #####################

# Import different packages
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np
import random

print('PyTorch Version',torch.__version__)
print('Numpy Version',np.__version__,'\n')
################################################################
torch.manual_seed(0)   #You can use torch.manual_seed() to seed the RNG for all devices (both CPU and CUDA)
random.seed(0)         #For custom operators, you might need to set python seed
np.random.seed(0)      #If you or any of the libraries you are using rely on NumPy, you can seed the global NumPy RNG
################################################################

# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    best_loss = np.Inf
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Save best model based on loss
            if loss.detach().numpy() < best_loss:
                print('Loss improved from {:.2f} to {:.2f}'.format(best_loss,loss.detach().numpy()))
                best_loss = loss.detach().numpy()
                
                torch.save(model, 'best_linreg_model.pth')
            
            # 4. Compute gradients
            loss.backward()
            
            # 5. Update parameters using gradients
            opt.step()
            
            # 6. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) == num_epochs:
            #print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            print('\nBest Loss: {:.2f}\n'.format(best_loss))
            model = torch.load('best_linreg_model.pth')
            
            for key in model.state_dict():
                print('key: ', key)
                param = model.state_dict()[key]
                print(param)

                
######################################################################            
# Input (x, y, z)
inputs = np.array([[75, 63, 48], 
                   [91, 88, 66], 
                   [84, 137, 57], 
                   [108, 41, 36], 
                   [68, 98, 72]], dtype='float32')

# Targets (a, b)
targets = np.array([[54, 71], 
                    [84, 111], 
                    [122, 143], 
                    [20, 40], 
                    [101, 117]], dtype='float32')

# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets,'\n')


#########################################################################
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

# Define data loader
batch_size = 3  # should be << number of observations
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

for xb, yb in train_dl:
    print(xb)
    print(yb,'\n')
    break

    
###########################################################################
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias,'\n')

# Define loss function
loss_fn = F.mse_loss

# SGD Optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

fit(100, model, loss_fn, opt, train_dl)

PyTorch Version 1.5.1+cu101
Numpy Version 1.18.5 

tensor([[ 75.,  63.,  48.],
        [ 91.,  88.,  66.],
        [ 84., 137.,  57.],
        [108.,  41.,  36.],
        [ 68.,  98.,  72.]])
tensor([[ 54.,  71.],
        [ 84., 111.],
        [122., 143.],
        [ 20.,  40.],
        [101., 117.]]) 

tensor([[108.,  41.,  36.],
        [ 91.,  88.,  66.],
        [ 75.,  63.,  48.]])
tensor([[ 20.,  40.],
        [ 84., 111.],
        [ 54.,  71.]]) 

Parameter containing:
tensor([[-0.0114,  0.4578, -0.0512],
        [ 0.1528, -0.1745, -0.1135]], requires_grad=True)
Parameter containing:
tensor([-0.5516, -0.3824], requires_grad=True) 

Loss improved from inf to 7459.46
Loss improved from 7459.46 to 5486.36
Loss improved from 5486.36 to 3183.69
Loss improved from 3183.69 to 1652.11
Loss improved from 1652.11 to 937.11
Loss improved from 937.11 to 636.37
Loss improved from 636.37 to 347.09
Loss improved from 347.09 to 46.10
Loss improved from 46.10 to 44.47
Loss improved from 44.47 to

In [7]:
# Generate predictions
model = torch.load('best_linreg_model.pth')
model.eval()

preds = model(inputs)
preds

tensor([[ 55.9501,  74.3751],
        [ 80.4069, 103.1202],
        [125.4135, 142.2679],
        [ 27.5171,  52.3162],
        [ 94.7545, 113.0206]], grad_fn=<AddmmBackward>)

In [8]:
# Actual outputs
targets

tensor([[ 54.,  71.],
        [ 84., 111.],
        [122., 143.],
        [ 20.,  40.],
        [101., 117.]])

In [9]:
# Inferencing a given input
model(torch.tensor([[72, 65, 42.]]))  #testing

tensor([[57.0600, 73.8723]], grad_fn=<AddmmBackward>)

In [10]:
# Verify the best model's weights and bias
for key in model.state_dict():
    print('key: ', key)
    param = model.state_dict()[key]
    print(param)

key:  weight
tensor([[-0.1638,  0.9333,  0.2080],
        [ 0.0399,  0.8696,  0.3537]])
key:  bias
tensor([-0.5510, -0.3802])
